# Neo4j
***
## Importing Libraries

In [1]:
from neo4j_functions import ne04j

## Creating a Movie Node to check the connection

In [2]:
ne04j.current_database = 'neo4j'

In [3]:
query = '''
MATCH (movie:Movie {title:$favorite})<-[:ACTED_IN]-(actor)-[:ACTED_IN]->(rec:Movie)
 RETURN distinct rec.title as title LIMIT 20
'''

for record in ne04j.get('MATCH (movie:Movie) RETURN movie LIMIT 5',
                        {"favorite": "The Matrix"}):
  print(record)


{'movie': {'tagline': 'Welcome to the Real World', 'title': 'The Matrix', 'released': 1999}}
{'movie': {'tagline': 'Free your mind', 'title': 'The Matrix Reloaded', 'released': 2003}}
{'movie': {'tagline': 'Everything that has a beginning has an end', 'title': 'The Matrix Revolutions', 'released': 2003}}
{'movie': {'tagline': 'Evil has its winning ways', 'title': "The Devil's Advocate", 'released': 1997}}
{'movie': {'tagline': "In the heart of the nation's capital, in a courthouse of the U.S. government, one man will stop at nothing to keep his honor, and one will stop at nothing to find the truth.", 'title': 'A Few Good Men', 'released': 1992}}


## Creating a Graph Database

In [4]:
pharm_db = """
// Creating nodes
CREATE (p1:Pharmacy {name: 'Pharmacy 1', address: 'Street 1'})
CREATE (p2:Pharmacy {name: 'Pharmacy 2', address: 'Street 2'})

CREATE (m1:Medicine {name: 'Medicine 1', price: 100})
CREATE (m2:Medicine {name: 'Medicine 2', price: 200})

CREATE (r1:Reservation {date: '2024-10-30', quantity: 2})
CREATE (r2:Reservation {date: '2024-10-31', quantity: 1})

CREATE (mf1:Manufacturer {name: 'Manufacturer 1'})
CREATE (mf2:Manufacturer {name: 'Manufacturer 2'})

CREATE (s1:Sales {date: '2024-10-30', quantity: 3})
CREATE (s2:Sales {date: '2024-10-31', quantity: 1})

CREATE (i1:Inventory {quantity: 10})
CREATE (i2:Inventory {quantity: 5})

// Creating relationships
CREATE (p1)-[:HAS]->(i1)
CREATE (p2)-[:HAS]->(i2)

CREATE (mf1)-[:PRODUCES]->(m1)
CREATE (mf2)-[:PRODUCES]->(m2)

CREATE (p1)-[:SELLS]->(s1)
CREATE (p2)-[:SELLS]->(s2)

CREATE (p1)-[:RESERVES]->(r1)
CREATE (p2)-[:RESERVES]->(r2)

CREATE (i1)-[:CONTAINS]->(m1)
CREATE (i2)-[:CONTAINS]->(m2)

CREATE (s1)-[:SELLS]->(m1)
CREATE (s2)-[:SELLS]->(m2)

CREATE (r1)-[:RESERVES]->(m1)
CREATE (r2)-[:RESERVES]->(m2)
"""

In [5]:
ne04j.create_new_db(pharm_db, 'Pharmacy')

ClientError: {code: Neo.ClientError.Database.ExistingDatabaseFound} {message: Failed to create the specified database 'Pharmacy': Database name or alias already exists.}

## Checking the Database List

In [6]:
ne04j.db_list

['neo4j', 'nosqllnu', 'pharmacy', 'system']

## Lets work on the Pharmacy Database

In [7]:
ne04j.current_database = 'Pharmacy'

## Querying the Database

- ### Query 1: Get all the Pharmacies

In [8]:
query = '''
MATCH (p:Pharmacy)
RETURN p.name, p.address
'''

In [9]:
for record in ne04j.get(query):
  print(record)

{'p.name': 'Pharmacy 1', 'p.address': 'Street 1'}
{'p.name': 'Pharmacy 2', 'p.address': 'Street 2'}


- ### Query 2: Get all the Medicines

In [10]:
query = '''
MATCH (m:Medicine)
RETURN m.name, m.price
'''

In [11]:
for record in ne04j.get(query):
  print(record)

{'m.name': 'Medicine 1', 'm.price': 100}
{'m.name': 'Medicine 2', 'm.price': 200}


- ### Query 3: Get all the Reservations

In [12]:
query = '''
MATCH (r:Reservation)
RETURN r.date, r.quantity
'''

In [13]:
for record in ne04j.get(query):
  print(record)

{'r.date': '2024-10-30', 'r.quantity': 2}
{'r.date': '2024-10-31', 'r.quantity': 1}


- ### Query 4: Update the Inventory

In [14]:
query = '''
MATCH (i:Inventory)
SET i.quantity = i.quantity + 5
RETURN i.quantity
'''

In [20]:
print("Old Inventory:", ne04j.get('MATCH (i:Inventory) RETURN i.quantity'))
ne04j.run(query)
print("Updated Inventory:", ne04j.get('MATCH (i:Inventory) RETURN i.quantity'))

Old Inventory: [{'i.quantity': 45}, {'i.quantity': 40}]
Updated Inventory: [{'i.quantity': 50}, {'i.quantity': 45}]


- ### Query 5: Create a Pharmacy 3

In [23]:
query = '''
CREATE (p:Pharmacy {name: 'Pharmacy 3', address: 'Street 3'})
'''

In [24]:
print("Before Creation:" , ne04j.get('MATCH (p:Pharmacy) RETURN p.name'))
ne04j.run(query)
print("After Creation:" , ne04j.get('MATCH (p:Pharmacy) RETURN p.name'))

Before Creation: [{'p.name': 'Pharmacy 2'}]
After Creation: [{'p.name': 'Pharmacy 3'}, {'p.name': 'Pharmacy 2'}]


- ### Query 6: Delete a Pharmacy 1

In [27]:
query = '''
MATCH (p:Pharmacy {name: 'Pharmacy 3'})
DETACH DELETE p
'''

In [28]:
print("Before Deletion:" , ne04j.get('MATCH (p:Pharmacy) RETURN p.name'))
ne04j.run(query)
print("After Deletion:" , ne04j.get('MATCH (p:Pharmacy) RETURN p.name'))

Before Deletion: [{'p.name': 'Pharmacy 3'}, {'p.name': 'Pharmacy 2'}]
After Deletion: [{'p.name': 'Pharmacy 2'}]


- ### Query 7: Get all the Pharmacies with their Medicines

In [29]:
query = '''
MATCH (p:Pharmacy)-[:HAS]->(i:Inventory)-[:CONTAINS]->(m:Medicine)
RETURN p.name, m.name
'''

In [30]:
for record in ne04j.get(query):
  print(record)

{'p.name': 'Pharmacy 2', 'm.name': 'Medicine 2'}
